In [1]:
import numpy as np 
import pandas as pd 
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import subprocess
import re
from tqdm import tqdm
tqdm.pandas()

import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from logger import logger

In [2]:
# url = 'https://www.imdb.com/title/tt0111161/?ref_=sr_i_1'

url = 'https://www.imdb.com/title/tt10888594/?ref_=sr_i_1'

In [3]:
driver = webdriver.Chrome()


In [4]:
reviews_link = url

In [5]:
# got movie title and link

driver.get(reviews_link)
time.sleep(3)

In [6]:
click_reviews = driver.find_element(By.XPATH, "//div[@data-testid='reviews-header']")
reviews_link = click_reviews.find_element(By.TAG_NAME,'a').get_attribute('href')

In [7]:
driver.get(reviews_link)

In [8]:
reviews_element = driver.find_element(By.XPATH, "//div[@data-testid='tturv-total-reviews']")
reviews_text = reviews_element.text
# Use regex to extract only the numeric part
reviews_count = int(re.sub(r"[^\d]", "", reviews_text))

print(reviews_count)

2577


In [9]:
def click_all_button():
    #  click button to show all reviews
    try:
        # Locate the correct <span> first, then find the button inside it
        span_element = driver.find_element(By.XPATH, "//span[contains(@class, 'chained-see-more-button')]")
        button = span_element.find_element(By.TAG_NAME, "button")

        if button:
            # Click the button using ActionChains (sometimes needed for better interaction)
            ActionChains(driver).move_to_element(button).click().perform()
        
            print("Correct 'All' button clicked successfully.")
    except Exception as e:
        print(f"Error: {e}")

In [10]:
max_wait_time = 600  
polling_interval = 1.5  # Check every 1.5 seconds
target_count = reviews_count  # Expected number of articles

start_time = time.time()

click_all_button()

while True:
    # Count current number of loaded articles
    articles = driver.find_elements(By.TAG_NAME, "article")
    current_count = len(articles)

    print(f"Loaded articles: {current_count}")

    # If we reach the target, break the loop
    if current_count >= target_count:
        print("All articles loaded!")
        break

    # Stop waiting if max wait time is exceeded
    if time.time() - start_time > max_wait_time:
        print("Timeout reached. Extracting available articles.")
        break
    

    if articles:
        last_article = articles[-1]  # Get the last article
        driver.execute_script("arguments[0].scrollIntoView();", last_article)
        # print("Scrolled to the last article.")
    else:
        # print("No articles found.")
        pass
    
    click_all_button()
    
    time.sleep(polling_interval)  # Wait before checking again


Correct 'All' button clicked successfully.
Loaded articles: 25
Correct 'All' button clicked successfully.
Loaded articles: 150
Correct 'All' button clicked successfully.
Loaded articles: 375
Correct 'All' button clicked successfully.
Loaded articles: 575
Correct 'All' button clicked successfully.
Loaded articles: 750
Correct 'All' button clicked successfully.
Loaded articles: 874
Correct 'All' button clicked successfully.
Loaded articles: 999
Correct 'All' button clicked successfully.
Loaded articles: 1174
Correct 'All' button clicked successfully.
Loaded articles: 1324
Correct 'All' button clicked successfully.
Loaded articles: 1499
Correct 'All' button clicked successfully.
Loaded articles: 1649
Correct 'All' button clicked successfully.
Loaded articles: 1824
Correct 'All' button clicked successfully.
Loaded articles: 1999
Correct 'All' button clicked successfully.
Loaded articles: 2174
Correct 'All' button clicked successfully.
Loaded articles: 2349
Correct 'All' button clicked succ

In [11]:
articles[0]

<selenium.webdriver.remote.webelement.WebElement (session="cc298184fdaf07a7dd454bbc13fe3825", element="f.0C1554D1A7B74737C74DC1C10BF126CA.d.5AA353242F44C3E596290448EC9A0C03.e.130")>

In [12]:
review_star = None


try:
    review_star =  articles[0].find_element(By.CLASS_NAME,'ipc-rating-star--rating').text
except:
    review_star = None
    
review_star

'1'

In [13]:
max_star = None
try:
    max_star =  articles[0].find_element(By.CLASS_NAME,'ipc-rating-star--maxRating').text[1:]
except:
    max_star = None
    
max_star

'10'

In [19]:
review_title = None
try:
    review_title =  articles[1].find_element(By.CLASS_NAME,'ipc-title__text').text
except:
    review_title = None
    
review_title

'Trash of the year'

In [15]:
By.XPATH, "//div[@data-testid='tturv-total-reviews']"

('xpath', "//div[@data-testid='tturv-total-reviews']")

In [18]:
review_text = None
try:
    review_text =  articles[3].find_element(By.XPATH,"//div[@data-testid='review-overflow']").text
except:
    review_text = None
    
review_text

'Salman has competition from himself to make worst movie.\n\nAnd he won this time too.'

In [21]:
# ipc-overflowText--children

review_text = None
try:
    review_text =  articles[7].find_element(By.CLASS_NAME,"ipc-overflowText--children").text
except:
    review_text = None
    
review_text

'No suspense , No thriller , Zero storyline.\n\nSalman khan is a feast to his fans with his ever charming looks. But the movie completely lacks in story, screenplay, direction,etc...'

In [17]:
    # review_text = None
    # try:
    #     review_text =  article.find_element(By.XPATH,"//div[@data-testid='review-overflow']").text
    # except:
    #     review_text = None

In [24]:
sample_articles = articles[:100]

In [25]:
review_stars = []
max_stars = []
review_titles = []
review_texts = []



for article in tqdm(sample_articles):
    
    review_star = None
    try:
        review_star =  article.find_element(By.CLASS_NAME,'ipc-rating-star--rating').text
    except:
        review_star = None
        
    review_stars.append(review_star)
        
    
        
    max_star = None
    try:
        max_star =  article.find_element(By.CLASS_NAME,'ipc-rating-star--maxRating').text[1:]
    except:
        max_star = None
    max_stars.append(max_star)
        
    review_title = None
    try:
        review_title =  article.find_element(By.CLASS_NAME,'ipc-title__text').text
    except:
        review_title = None
    
    review_titles.append(review_title)
        

    
    # review_text = None

    # try:
    #     # Check if Spoiler button exists
    #     spoiler_button = article.find_element(By.XPATH, "//button[contains(@class, 'review-spoiler-button')]")
    #     if spoiler_button:
    #         review_text = None  # Set to None if spoiler button is found
    #     else:
    #         review_text = article.find_element(By.XPATH, "//div[@data-testid='review-overflow']").text
    # except NoSuchElementException:
    #     # If no spoiler button is found, try extracting review
    #     try:
    #         review_text = article.find_element(By.XPATH, "//div[@data-testid='review-overflow']").text
    #     except NoSuchElementException:
    #         review_text = None  # Set to None if review is also not found
    
    
    review_text = None
    try:
        review_text =  article.find_element(By.CLASS_NAME,"ipc-overflowText--children").text
    except:
        review_text = None
            
    review_texts.append(review_text)
        
    

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:18<00:00,  5.36it/s]


In [26]:
df = pd.DataFrame({
    "review_star": review_stars,
    "max_star": max_stars,
    "review_title": review_titles,
    "review_text": review_texts
})

df

,review_star,max_star,review_title,review_text
0,1,10,Donate money for Corona patients rather to was...,Salman has competition from himself to make wo...
1,1,10,Trash of the year,"Storyline Trash, Acting of Salman khan Trash, ..."
2,1,10,As always.,This is the best movie if you want to nit watc...
3,1,10,Worst movie of all the time,Salman Khan needs to take retirement. This mov...
4,1,10,Maha Bakwassss...,I would rather prefer watching TVF webseries a...
...,...,...,...,...
95,1,10,Hyped for no reason,It's not new first Bollywood to adapt or in th...
96,1,10,"All brawn, no brain! At best a polished, medio...","Your ""Most Wanted Bhai"" is here, in your drawi..."
97,9,10,MUST WATCH,Fast paced movie. Not at all boring. Good mess...
98,None,None,Trash movie,Don't waste your time on this movie.\n\nMovie ...
